# 案例 1.2 北京奥运会奖牌的分布及构成分析

第 29 届夏季奥林匹克运动会于 2008 年 8 月 8 日至 8 月 24 日在中国北京举行，来自 204 个国家和地区的 11028 名运动员参加了 28 个大项，共 302 项赛事，其中：86 个代表团赢得了奖牌，54 个代表团赢得了至少 1 枚金牌。

无论是获得金牌的代表团数量还是获得奖牌的代表团数量，这届奥运会都创下了新的纪录。东道主中国队一共赢得 48 枚金牌（原金牌数为 51 枚，后因禁药问题取消了其中 3 枚），名列第一，这也是中国在夏季奥运会历史上首次位居金牌榜榜首。美国队获得的奖牌总数最多，有 112 枚。来自阿富汗、毛里求斯、苏丹、塔吉克斯坦和多哥代表团的运动员获得了该国历史上的第一枚夏季奥运会奖牌。来自蒙古和巴拿马代表团的运动员则赢得了该国历史上的第一枚夏季奥运会金牌。塞尔维亚游泳运动员米洛拉德·查维奇为该国赢得了一枚银牌，成为该国以独立国家名义获得的首枚奥运会奖牌。塞尔维亚在历史上曾作为南斯拉夫或塞尔维亚和黑山的一部分参加奥运会并赢得过奖牌。萨摩亚在 2016 年药检后的奖牌重分配中获得了历史上首枚奖牌。

教材给出的数据只是三个国家奖牌的汇总数据，并不是原始数据，原始数据可通过爬取[维基百科](https://en.wikipedia.org/wiki/2008_Summer_Olympics_medal_table)页面获取。

## 1 数据爬取

数据的爬取包括请求页面、提取数据和存储数据三个步骤，我们将使用 Python 第三方库 urllib 请求页面, 使用 Python 第三方库 lxml 和标准库 re 提取数据，使用 Python 第三方库 numpy, pandas 存储数据。

### 1.1 爬取前的准备

爬取各国家具体获奖信息前的准备工作包括：

* 设置代理
* 从“维基百科:2008 夏季奥运会奖牌表”页面中提取所有获奖国家的页面地址
* 初始化存储数据的数组、存储爬取失败页面的列表以及存储爬取有问题页面的列表

以下是 Python 代码

In [ ]:
from urllib.request import ProxyHandler, build_opener, \
                            install_opener, Request, urlopen
from urllib.error import URLError
from lxml import etree
import re
import numpy as np
import pandas as pd


# ---1.1 preparation---

# set proxy
proxy_handler = ProxyHandler({'http': 'http://127.0.0.1:57858',
                              'https': 'https://127.0.0.1:57858'})
opener = build_opener(proxy_handler)
install_opener(opener)

# get and set page url
publicURL = 'https://en.wikipedia.org'
year = '2008'
url = 'https://en.wikipedia.org/wiki/' + year + '_Summer_Olympics_medal_table'
# set UA，disguise crawler as browser
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
          + "AppleWebKit/537.36 (KHTML, like Gecko) "
          + "Chrome/77.0.3865.90 Safari/537.36"}
response = urlopen(Request(url, headers=header))
html = etree.HTML(response.read())
privateURL_list = html.xpath('//table[@class="wikitable sortable '
                             + 'plainrowheaders jquery-tablesorter"]'
                             + '/tbody/tr//a/@href')
countryList = list(map(lambda x: re.sub('/wiki/|_at.*', '', x),
                       privateURL_list))

# initialize a ndarray athData for athletes' data storage
# initialize a list for crawler failure page
# initialize a list for crawler problem page
athData, errorList, problemList = np.zeros([1, 5]), [], []


接着爬取获奖国家的页面，爬取的数据存储于 athData 对象中

In [ ]:
# ---1.2 crawling---

def proc_data(data, data_header, country):
    data = list(map(lambda x: re.sub(r'\[\d\]|/|.*\n', '\n', x), data))
    data = list(filter(lambda x: x != '\n' and x != '*' and
                       x != '\xa0' and x != ' ', data))

    # merge names of athletes for team events
    end = 2 - len(data_header)
    temp = data[1:end]
    if len(temp) > 1:
        athNameSet = ', '.join(temp)
        del data[2:end]
        data[1] = athNameSet

    data = list(map(lambda x: re.sub('\xa0|\n', '', x), data))
    if len(data) > 4:
        data = data[0:4]
    data.append(country)
    return np.array([data])


def select_property_name(html, country):
    data = html.xpath('(//table[contains(@class, "wikitable sortable"'
                      + ')])[1]/tbody/tr[2]/td//text()')
    data_header = html.xpath('(//table[contains(@class, "wikitable sortable"'
                             + ')])[1]/tbody/tr/th/text()')
    data = proc_data(data, data_header, country)
    if data.size == 5:
        return '"wikitable sortable"'
    else:
        return '"wikitable"'

print('\nCrawling start...\n')

# request, extract and store pages
for i, privateURL in enumerate(privateURL_list):
    has_problem = False
    athURL = publicURL + privateURL
    print('------ requesting {0}th page，{1} pages are remaining ------\n'
          .format(i + 1, len(privateURL_list) - i - 1))
    print('[URL]:     {0}'.format(athURL))

    for request_num in range(1, 6):
        try:
            response = urlopen(Request(athURL, headers=header))
            html = etree.HTML(response.read())
        except Exception as e:
            if request_num < 5:
                print(('[Warning]:   {0}th request was failed，'
                      + 'trying {1}th request...')
                      .format(request_num, request_num + 1))
            else:
                errorList.append(athURL)
                print('[Error]:   two much failure，requesting '
                      + 'next page soon...\n')
        else:
            print('[Success]: extracting data from page...')
            country = countryList[i]
            propertyName = select_property_name(html, country)
            athDataHeader = html.xpath('(//table[contains(@class, '
                                       + propertyName + ')])[1]'
                                       + '/tbody/tr/th/text()')
            # extract data of page row by row
            j = 2
            while True:
                str_j = str(j)
                has_s = html.xpath('(//table[contains(@class, ' + propertyName
                                   + ')])[1]/tbody/tr[' + str_j + ']/td/s')
                rowData = html.xpath('(//table[contains(@class, '
                                     + propertyName + ')])[1]/tbody/tr['
                                     + str_j + ']/td//text()')
                # if <s></s> exits，ignore current rowData
                # <s> represents strickout
                if len(has_s) > 0:
                    j += 1
                    continue
                # current page's data was extracted when rowData is null
                # so stop while iteration
                elif len(rowData) == 0:
                    break
                else:
                    rowData = proc_data(rowData, athDataHeader, country)
                    # add current page's data to athData
                    try:
                        athData = np.concatenate((athData, rowData), axis=0)
                    except ValueError as e:
                        problemList.append(athURL)
                        has_problem = True
                    else:
                        j += 1

            if has_problem is True:
                print('[Warning]: current page may be incompatible with rule'
                      + ' of crawler, check manually')
                print('[Error]:   incomplete data extraction，'
                      + 'requesting next page soon...\n')
            elif i < len(privateURL_list) - 1:
                print('[Success]: data extraction was done，'
                      + 'requesting next page soon...\n')
            else:
                print('[Success]: data extraction was done\n')

            break

if len(errorList) > 0:
    print('[Warning]: following pages\' crawling are failed！\n')
    for item in errorList:
        print(item)
    print('\nCheck network, program and web pages carefully, '
          + 'failure is the mother of success （づ￣3￣）づ╭❤～\n')
elif len(problemList) > 0:
    print('[Warning]: following pages\' crawling are incomplete！\n')
    for item in problemList:
        print(item)
    print('\nCheck network, program and web pages carefully, '
          + 'failure is the mother of success （づ￣3￣）づ╭❤～\n')
else:
    print('Congratulations to you, all are successful（づ￣3￣）づ╭❤～\n')

最后将 athData 数组转换为 pandas 中的 DataFrame 类，并利用 DataFrame 类方法 to_execl() 将数据输出为外部 xlsx 文件

In [ ]:
# ---1.3 output data---

athData = athData[1:]
athDF = pd.DataFrame(athData, columns=['mdal_type', 'name', 'sport',
                                       'discipline', 'country'])
athDF.to_excel('F:/web_crawler_result/Olympic/medalInfo' + year + '.xlsx',
               sheet_name=year)

使用 athDF.head(10) 查看该数据表前 10 行

In [ ]:
athDF.head(10)

通过对项目作分词处理，我们可以获得性别变量。大家可以看到：奖牌类型、姓名、运动、项目、国家、性别，它们均是分类型数据。通过这张数据表，我们可以获得许多信息，例如：

* 对单个分类型变量取值的了解（唯一值，6）
  + 有哪些运动员（获奖）
  + 有哪些运动
  + 各运动有哪些项目
  + 哪些国家获得了奖牌
* 对单个分类型变量取值频数/频率的了解（频数表，6）
  + 金银铜牌比
  + 哪些运动奖牌多
  + 哪些国家奖牌多
  + （获奖运动员）男女比
* 对两个分类型变量取值频数/频率的了解（二向列联表，$\binom{6}{2}=15$）
  + 运动员奖牌榜
  + 运动奖牌榜
  + 国家奖牌榜
  + 性别奖牌榜
  + 各国家（获奖）运动员男女比
* 对两个分类型变量是否相关的了解（列联表独立性检验，15）
  + 各国家的奖牌类型分布类似吗 / 各奖牌类型的国家分布类似吗 / 奖牌类型与国家独立吗
* 对三个分类型变量取值频数/频率的了解（三向列联表，$\binom{6}{3}=20$）
  + 各运动国家奖牌榜
  + 各运动运动员奖牌榜
  + 各运动性别奖牌榜
* 对三个分类型变量是否相关的了解（列联表独立性检验，20）
  + 各国家各性别的奖牌类型分布类似吗 / 各国家各奖牌类型的性别分布类似吗 / 奖牌类型、国家、性别独立吗

限于时间，笔者没有对该原始数据作后续处理和深入分析。下面我们将直接从部分汇总数据（金牌榜排名前 10 的国家的奖牌榜）出发。这个汇总数据就是二向列联表，由国家和奖牌类型两个分类型变量构成。

|   国家   |   金   |   银    |   铜    |  总 |
|  :---:  |  :---:  |  :---:  |  :---:  |  :---:  |
|   中国  |    48   |   22    |    30   |  100  |
|   美国  |    36   |   39    |    37   |  112 |
|  俄罗斯  |    22   |   15    |    23   |  60  |
|   英国  |    19   |   13    |    15   |  47  |
|   德国  |    16   |   10    |    15   |  41 |
| 澳大利亚  |    14   |   15    |    17   |  46 |
|   韩国  |    13   |   11    |    8   |  32  |
|   日本  |    9   |   7    |    9   |  25  |
|  意大利  |    8   |   9    |    10   |  27 |
|   法国  |    7   |   16    |    19   | 42  |
|   其它  |    107   |   142    |    165   |  414 |
|   总计  |   302	   |   303   |	 353	 |   958  |

下面，将结合上述汇总数据介绍如何使用 Python 第三方库 <a href="https://pyecharts.org/#/zh-cn/intro">pyecharts</a> 绘制条形图、饼图和雷达图。

<a href="https://echarts.apache.org/zh/index.html">Echarts</a> 是一个由百度开源的，使用 JavaScript 编写的数据可视化库，凭借着良好的交互性，精巧的图表设计，得到了众多开发者和客户的认可，目前 Echarts 在 <a href="https://github.com/apache/incubator-echarts">Github</a> 上的标星数达到了 36299，其使用者包括国家统计局、国家电网、中石化、中国人民保险、华为、阿里、腾讯、百度、新浪、网易、小米、滴滴等许多知名企业和大型机构。

pyecharts 是由 <a href="https://chenjiandongx.com/">陈健冬</a> 开发，由 <a href="https://github.com/chenjiandongx">@chenjiandongx</a>、<a href="https://github.com/chfw">@chfw</a>、<a href="https://github.com/kinegratii">@kinegratii</a> 和 <a href="https://github.com/sunhailin-Leo">@sunhailin-Leo</a> 等维护的 Python 第三方数据可视化库。利用 pyecharts，使用 Python 就可以绘制出 Echarts 提供的许多漂亮的、带交互功能的图，目前该项目 Github 标星数为 6751.

## 2 刻画分类型数据的图表

刻画分类变量数据的表为频数/频率表（frequency table），刻画两个以上分类变量数据的频数/频率表又称为列联表/交叉表（contingency/cross table），通过列联表，利用皮尔逊卡方检验等方法，可以检验两个或多个分类变量的独立性，计算 Phi 系数、Carmer V 系数或列联系数等可以定量衡量两个或多个分类变量的相关性。

刻画单个分类型变量数据的图有

* 条形图（bar plot）
* 帕累托图（pareto plot）：条形图按从小到大排序，并添加累积频数线
* 饼图（pie chart）
* 扇图（fan chart）：饼图的改进版
* ……

刻画两个分类型变量数据的图有

* 复式条形图（double bar plot）：条形图的推广
* 环形图（ring plot）：饼图的推广
* 脊形图（spine plot）
* 雷达图（radar chart）
* ……

刻画三个以上分类型变量数据的图有

* 马赛克图（mosaic plot）：脊形图的推广
* ……

下面要讲的各国家的奖牌类型分布以及各奖牌类型的国家分布，均涉及两个分类型变量国家和奖牌类型。因此，我们只讲如何利用 Python 包 pyecharts 画两个分类型变量数据的（复式）条形图，环形图和雷达图。需要指出的是，pyecharts 没有提供帕累托图、扇图、脊形图和马赛克图。

## 3 各国家的奖牌类型分布

### 3.1 复式条形图

In [ ]:
import numpy as np
from pyecharts.charts import Bar
from pyecharts import options as opts


dfValue = np.array([[48, 22, 30],
                    [36, 39, 37],
                    [22, 15, 23],
                    [19, 13, 15],
                    [16, 10, 15],
                    [14, 15, 17],
                    [13, 11,  8],
                    [ 9,  7,  9],
                    [ 8,  9, 10],
                    [ 7, 16, 19],
                    [107,142,165]])
dfIndex = ['中国', '美国', '俄罗斯', '英国', '德国',
           '澳大利亚', '韩国', '日本', '意大利', '法国', '其它']
dfColumns = ['金', '银', '铜']

dfValueT = dfValue.T

bar = (
    Bar()
    .add_xaxis(dfIndex)
    .add_yaxis(dfColumns[0], dfValueT[0].tolist(), gap=0)
    .add_yaxis(dfColumns[1], dfValueT[1].tolist(), gap=0)
    .add_yaxis(dfColumns[2], dfValueT[2].tolist(), gap=0)
    .set_global_opts(
        title_opts=opts.TitleOpts(title='2008 年北京奥运会各国的奖牌分布',subtitle='并列条形图（仅包含金牌榜前 10 国家）'),
        legend_opts=opts.LegendOpts(pos_left='center'),
        yaxis_opts=opts.AxisOpts(name="奖牌数", name_location='center', name_rotate=90, name_gap=35),
        xaxis_opts=opts.AxisOpts(name="国家", name_location='center', name_gap=35),
        datazoom_opts=opts.DataZoomOpts(type_='inside', range_start=0, range_end=90),
        toolbox_opts=opts.ToolboxOpts()
    )
)

print('\n')
bar.render_notebook()

In [ ]:
bar = (
    Bar()
    .add_xaxis(dfIndex)
    .add_yaxis(dfColumns[0], dfValueT[0].tolist(), stack='stack1', gap=0)
    .add_yaxis(dfColumns[1], dfValueT[1].tolist(), stack='stack1', gap=0)
    .add_yaxis(dfColumns[2], dfValueT[2].tolist(), stack='stack1', gap=0)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='2008 年北京奥运会各国的奖牌分布',subtitle='堆叠条形图（仅包含金牌榜前 10 国家）'),
        legend_opts=opts.LegendOpts(pos_left='center'),
        yaxis_opts=opts.AxisOpts(name="奖牌数", name_location='center', name_rotate=90, 
                                 name_gap=35, max_=int(max(dfValue[:-1].sum(axis=1)))),
        xaxis_opts=opts.AxisOpts(name="国家", name_location='center', name_gap=35),
        datazoom_opts=opts.DataZoomOpts(type_='inside', range_start=0, range_end=90),
        toolbox_opts=opts.ToolboxOpts()
    )
)

print('\n')
bar.render_notebook()

从上面的（复式）条形图中，我们可以看到，就各国家的奖牌类型分布而言：

* 美国、澳大利亚、日本、意大利三种类型奖牌的数目基本一致
* 中国、俄罗斯、英国、德国三种类型奖牌的数目呈现出金牌铜牌较多、银牌较少的模式
* 比较特别的是韩国和法国，韩国铜牌数量相较金牌和银牌少很多，法国金牌数量相较银牌和铜牌少很多

下面通过环形图、雷达图进一步验证这一结论。

### 3.2 环形图

In [ ]:
from pyecharts.charts import Pie

dfValueP = dfValue * 100 / dfValue.sum(axis=1, keepdims=True)
round_vec = np.vectorize(round)
dfValueP = round_vec(dfValueP, 0)

pie = (
        Pie()
        .add(dfIndex[1], [list(z) for z in zip(dfColumns, dfValueP[1].tolist())], radius=["10%", "25%"])
        .add(dfIndex[5], [list(z) for z in zip(dfColumns, dfValueP[5].tolist())], radius=["26%", "41%"])
        .add(dfIndex[7], [list(z) for z in zip(dfColumns, dfValueP[7].tolist())], radius=["42%", "57%"])
        .add(dfIndex[8], [list(z) for z in zip(dfColumns, dfValueP[8].tolist())], radius=["58%", "73%"])
        .set_global_opts(
            title_opts=opts.TitleOpts(title="美澳日意四国的奖牌构成",subtitle='环形图'),
            #tooltip_opts=opts.TooltipOpts(formatter="{a}: \n{b}:{c}:%"),
            toolbox_opts=opts.ToolboxOpts()
        )
        #.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{c}%",position='inside'))
)

print('\n')
pie.render_notebook()

In [ ]:
pie = (
        Pie()
        .add(dfIndex[0], [list(z) for z in zip(dfColumns, dfValueP[0].tolist())], radius=["10%", "25%"])
        .add(dfIndex[2], [list(z) for z in zip(dfColumns, dfValueP[2].tolist())], radius=["26%", "41%"])
        .add(dfIndex[3], [list(z) for z in zip(dfColumns, dfValueP[3].tolist())], radius=["42%", "57%"])
        .add(dfIndex[4], [list(z) for z in zip(dfColumns, dfValueP[4].tolist())], radius=["58%", "73%"])
        .set_global_opts(
            title_opts=opts.TitleOpts(title="中俄英德四国的奖牌构成",subtitle='环形图'),
            toolbox_opts=opts.ToolboxOpts()
        )
        #.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{c}%",position='inside'))
)

print('\n')
pie.render_notebook()

In [ ]:
pie = (
        Pie()
        .add(dfIndex[6], [list(z) for z in zip(dfColumns, dfValueP[6].tolist())], radius=["40%", "55%"])
        .add(dfIndex[9], [list(z) for z in zip(dfColumns, dfValueP[9].tolist())], radius=["56%", "71%"])
        .set_global_opts(
            title_opts=opts.TitleOpts(title="韩法两国的奖牌构成",subtitle='环形图'),
            toolbox_opts=opts.ToolboxOpts()
        )
        #.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{c}%",position='inside'))
)

print('\n')
pie.render_notebook()

### 3.3 雷达图

In [ ]:
from pyecharts.charts import Radar


radar = (
          Radar()
          .add_schema(
                      schema=[
                              opts.RadarIndicatorItem(name="金", max_=50),
                              opts.RadarIndicatorItem(name="银", max_=50),
                              opts.RadarIndicatorItem(name="铜", max_=50),
                             ]
                     )
          .add(dfIndex[1], [dfValue[1].tolist()], color="#DC143C")
          .add(dfIndex[5], [dfValue[5].tolist()], color="#4169E1")
          .add(dfIndex[7], [dfValue[7].tolist()], color="#2E8B57")
          .add(dfIndex[8], [dfValue[8].tolist()], color="#FFA500")
          .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
          .set_global_opts(
              title_opts=opts.TitleOpts(title="美澳日意四国的奖牌构成",subtitle='雷达图'),
              toolbox_opts=opts.ToolboxOpts()
          )
)

print('\n')
radar.render_notebook()

In [ ]:
radar = (
          Radar()
          .add_schema(
                      schema=[
                              opts.RadarIndicatorItem(name="金", max_=50),
                              opts.RadarIndicatorItem(name="银", max_=50),
                              opts.RadarIndicatorItem(name="铜", max_=50),
                             ]
                     )
          .add(dfIndex[0], [dfValue[0].tolist()], color="#DC143C")
          .add(dfIndex[2], [dfValue[2].tolist()], color="#4169E1")
          .add(dfIndex[3], [dfValue[3].tolist()], color="#2E8B57")
          .add(dfIndex[4], [dfValue[4].tolist()], color="#FFA500")
          .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
          .set_global_opts(
              title_opts=opts.TitleOpts(title="中俄英德四国的奖牌构成",subtitle='雷达图'),
              toolbox_opts=opts.ToolboxOpts()
         )
)

print('\n')
radar.render_notebook()

In [ ]:
radar = (
          Radar()
          .add_schema(
                      schema=[
                              opts.RadarIndicatorItem(name="金", max_=20),
                              opts.RadarIndicatorItem(name="银", max_=20),
                              opts.RadarIndicatorItem(name="铜", max_=20),
                             ]
                     )
          .add(dfIndex[6], [dfValue[6].tolist()], color="#DC143C")
          .add(dfIndex[9], [dfValue[9].tolist()], color="#4169E1")
          .add(dfIndex[8], [dfValue[8].tolist()], color="#2E8B57",is_selected=False)
          .add(dfIndex[4], [dfValue[4].tolist()], color="#FFA500",is_selected=False)
          .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
          .set_global_opts(
              title_opts=opts.TitleOpts(title="韩法两国的奖牌构成",subtitle='雷达图'),
              toolbox_opts=opts.ToolboxOpts()
          )
)

print('\n')
radar.render_notebook()

## 4 各奖牌类型的国家分布

### 4.1 复式条形图

In [ ]:
bar = (
    Bar()
    .add_xaxis(dfColumns)
    .add_yaxis(dfIndex[0], dfValue[0].tolist(), gap=0)
    .add_yaxis(dfIndex[1], dfValue[1].tolist(), gap=0)
    .add_yaxis(dfIndex[2], dfValue[2].tolist(), gap=0)
    .add_yaxis(dfIndex[3], dfValue[3].tolist(), gap=0)
    .add_yaxis(dfIndex[4], dfValue[4].tolist(), gap=0)
    .add_yaxis(dfIndex[5], dfValue[5].tolist(), gap=0)
    .add_yaxis(dfIndex[6], dfValue[6].tolist(), gap=0)
    .add_yaxis(dfIndex[7], dfValue[7].tolist(), gap=0)
    .add_yaxis(dfIndex[8], dfValue[8].tolist(), gap=0)
    .add_yaxis(dfIndex[9], dfValue[9].tolist(), gap=0)
    .add_yaxis(dfIndex[10], dfValue[10].tolist(), gap=0, is_selected=False)
    .set_global_opts(
        title_opts=opts.TitleOpts(title='2008 年北京奥运会各奖牌的国家分布',subtitle='并列条形图'),
        legend_opts=opts.LegendOpts(pos_top=50, pos_right='right', orient='vertical'),
        yaxis_opts=opts.AxisOpts(name="奖牌数", name_location='center', name_rotate=90, name_gap=35),
        xaxis_opts=opts.AxisOpts(name="奖牌类型", name_location='center', name_gap=35),
        datazoom_opts=opts.DataZoomOpts(type_='inside', range_start=0, range_end=100),
        toolbox_opts=opts.ToolboxOpts()
    )
)

print('\n')
bar.render_notebook()

In [ ]:
bar = (
    Bar()
    .add_xaxis(dfColumns)
    .add_yaxis(dfIndex[0], dfValue[0].tolist(), stack='stack1', gap=0)
    .add_yaxis(dfIndex[1], dfValue[1].tolist(), stack='stack1', gap=0)
    .add_yaxis(dfIndex[2], dfValue[2].tolist(), stack='stack1', gap=0)
    .add_yaxis(dfIndex[3], dfValue[3].tolist(), stack='stack1', gap=0)
    .add_yaxis(dfIndex[4], dfValue[4].tolist(), stack='stack1', gap=0)
    .add_yaxis(dfIndex[5], dfValue[5].tolist(), stack='stack1', gap=0)
    .add_yaxis(dfIndex[6], dfValue[6].tolist(), stack='stack1', gap=0)
    .add_yaxis(dfIndex[7], dfValue[7].tolist(), stack='stack1', gap=0)
    .add_yaxis(dfIndex[8], dfValue[8].tolist(), stack='stack1', gap=0)
    .add_yaxis(dfIndex[9], dfValue[9].tolist(), stack='stack1', gap=0)
    .add_yaxis(dfIndex[10], dfValue[10].tolist(), gap=0)
    .reversal_axis()
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='2008 年北京奥运会各奖牌的国家分布',subtitle='并列堆叠条形图'),
        legend_opts=opts.LegendOpts(pos_top=50, pos_right='right', orient='vertical'),
        yaxis_opts=opts.AxisOpts(name="奖牌类型", name_location='center', name_rotate=90, 
                                 name_gap=35),
        xaxis_opts=opts.AxisOpts(name="奖牌数", name_location='center', name_gap=35, 
                                 max_=int(round(max(dfValue[:-1].sum(axis=0)),-1))+10),
        datazoom_opts=opts.DataZoomOpts(type_='inside', range_start=0, range_end=100, orient='vertical'),
        toolbox_opts=opts.ToolboxOpts()
    )
)

print('\n')
bar.render_notebook()

从上面的（复式）条形图中，我们可以看到，就各奖牌类型的国家分布而言：

* 金牌的国家分布与银牌或铜牌的国家分布存在较大差异
* 银牌的国家分布和铜牌的国家分布差异较小
* 金牌榜前十国家的奖牌占有率超过 50%，特别是金牌占有率远超 50%

下面通过环形图进一步验证这一结论。

### 4.2 环形图

In [ ]:
dfValueTP = dfValue.T * 100 / dfValue.T.sum(axis=1, keepdims=True)
round_vec = np.vectorize(round)
dfValueTP = round_vec(dfValueTP, 0)

pie = (
        Pie()
        .add(dfColumns[0], [list(z) for z in zip(dfIndex, dfValueTP[0].tolist())], radius=["30%", "45%"])
        .add(dfColumns[1], [list(z) for z in zip(dfIndex, dfValueTP[1].tolist())], radius=["46%", "61%"])
        .add(dfColumns[2], [list(z) for z in zip(dfIndex, dfValueTP[2].tolist())], radius=["62%", "77%"])
        .set_global_opts(
            title_opts=opts.TitleOpts(title="金银铜牌的国家构成"),
            legend_opts=opts.LegendOpts(pos_top=50, pos_right='right', orient='vertical'),
            #tooltip_opts=opts.TooltipOpts(formatter="{a}: \n{b}:{c}:%"),
            toolbox_opts=opts.ToolboxOpts()
        )
        #.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{c}%",position='inside'))
)

print('\n')
pie.render_notebook()

**综上：各国家的奖牌类型分布存在一定的相似性和差异性，各奖牌类型的国家分布也存在一定的相似性和差异性。**